# mParticle Real Time Data + Personalize

In this module you are going to be adding the ability to maintain a real-time dataset that represents the latest user behavior for users of the Retail Demo Store.  You will then connect that dataset to the Personalize dataset groups that you built in the first part of the workshop. This will enable your Personalize models to be kept up to date with the latest events your users are performing.

This workshop will use the mParticle web sdk npm library (https://github.com/mParticle/mparticle-web-sdk) to collect real-time data from the Retail Demo Store, and then feed that event data into the mParticle platform, where it can be routed directly to a Personalize Tracker, and then used to maintain the latest behavioral data for your personalization user-item interaction data.

Recommended Time: 45 Minutes

## Prerequisites

In order to complete this workshop, you will need to complete the 1.1-Personalize workbook in this directory.  You will also need a mParticle workspace.  If you are doing this workshop as part of a live workshop event, ask your moderator how to set up a mParticle workspace.  If you are running this workshop on your own, you can go to https://www.mparticle.com/contact to request for the creation of a mParticle account.  We do not recommend using your production mParticle workspace for this workshop.

## mParticle Platform Overview
mParticle is a customer data platform (CDP) that helps you collect, clean, and control your customer data. mParticle provides several types of Sources which you can use to collect your data, and which you can choose from based on the needs of your app or site. For websites, you can use a javascript library to collect data. If you have a mobile app, you can embed one of mParticle’s Mobile SDKs, and if you’d like to create messages directly on a server (if you have, for example a dedicated .NET server that processes payments), mParticle has several server-based libraries that you can embed directly into your backend code. With mParticle, you can also use cloud-sources to import data about your app or site from other tools like Zendesk or Salesforce, to enrich the data sent through mParticle. By using mParticle to decouple data collection from data use, you can create a centralized data supply chain based on organized and modular data.

![mParticle Overview](images/mparticle/mparticle_overview.png)

## Setup

mParticle uses *connections* as a way to organize data inputs into the platform.  Configuring a input will allow you to collect real-time event data from the Retail Demo Store user interface, and pass that information to mParticle.  You need to be signed into your mParticle workspace to begin this process.  Once you are signed in to the mParticle console (https://app.mparticle.com), click on your workspace, and then ‘Setup’ in the left hand navigation bar of the screen. Then, click ‘Inputs’.

![mParticle Input Setup](images/mparticle/mparticle-step-1.png)

Select the ‘Web’ type within Platforms.

![mParticle Platform Catalog](images/mparticle/mparticle-step-2.png)

And click ‘Web’ then click Issue Keys.

![mParticle Input Setup Add JS Source](images/mparticle/mparticle-step-3.png)

mParticle will generate a pair of key and secret which you will use as part of the cloud formation template setup earlier.

![mParticle Input Setup Key_Secret](images/mparticle/mparticle-step-4.png)

Now that you are here, set the write key for your new source in the environment variable below.  

You will need this in a few minutes, when you enable mParticle events collection in the Retail Demo Store.

Make sure you run the cell after you paste the key.

In [1]:


# Enter your mParticle write key here, from the above step

mparticle_api_key = "YOUR_MPARTICLE_API_KEY_GOES_HERE"
mparticle_secret_key = "YOUR_MPARTICLE_SECRET_KEY_GOES_HERE"


Now that you have a working source, let’s wire up our event data to Amazon Personalize.

## Configure the mParticle Personalize Destination

mParticle uses Outputs to route real-time event data to a data consumer application.  In this case, you will be using the AWS Kinesis as the destination.  This destination will take real-time events from mParticle, pass them through an AWS Lambda function, and then into the user-item interactions dataset in your Retail Demo Store.

Prepare the AWS Assets.

For this use case I need to bring together mParticle and four AWS services:

    1.) A Kinesis stream receives events from mParticle
    2.) A Personalize campaign creates product recommendations
    3.) A Lambda function acts as a broker. It transforms data from mParticle into a format accepted by Personalize, and uploads product recommendations back to mParticle.
    4.) IAM controls access and permissions for the other components.
    
These services can be configured in the AWS UI, but I'll be using Amazon's CLI tool. This way, I can reuse my work by creating a script to quickly spin up future iterations. I've followed Amazon's documentation to create an IAM user with access to the above four systems and log in to the console.

As I go, I’ll need to save the Amazon Resource Number (ARN) for each asset I create. I’ll need these ARNs to set up interactions between the different resources I create.

### Create a Kinesis Stream

```
aws kinesis create-stream \ --stream-name Items4UCommerceEventStream \ --shard-count 1
```

Save the `StreamARN` from the response.

### Create a role for mParticle to assume

For mParticle to be able to upload to the Kinesis stream, I need to create an IAM role for mParticle to assume. This role needs a policy allowing PutRecord access to Kinesis (https://github.com/kale-bogdanovs/mparticle-and-amazon-personalize/blob/main/policy-samples/mp-kinesis-role.json), and a trust policy (https://github.com/kale-bogdanovs/mparticle-and-amazon-personalize/blob/main/policy-samples/mp-trust-policy.json) allowing mParticle to assume the role.

```
aws iam create-role --role-name mparticle-kinesis-role --assume-role-policy-document file:///path/to/mp-trust-policy.json

aws iam put-role-policy --role-name mparticle-kinesis-role --policy-name mp-kinesis-put --policy-document file:///path/to/mp-kinesis-role.json
```

### Connect mParticle to Kinesis

mParticle offers an "event" output for streaming event data to Kinesis in real time. This can be set up and controlled from the mParticle dashboard without writing code. You can read an overview of event outputs in the mParticle docs (https://docs.mparticle.com/guides/getting-started/connect-an-event-output/).

Kinesis is a tool for processing streaming data. mParticle will forward commerce event data to Kinesis, where it will be picked up by the Lambda function I'll set up later.

Click ‘Directory’ in the left hand navigation bar  of the screen, and then search ‘Amazon Kinesis’.

![mParticle Output Setup](images/mparticle/mparticle-step-5.png)

#### Create configuration

First, I need to create an overall configuration for Kinesis. This holds all the settings that will remain the same for every input I connect. Each mParticle integration requires different settings. For example, API keys are commonly required. For Kinesis, I've already granted mParticle write access using IAM, so I only need to provide my AWS account number here.

![mParticle Kinesis Setup1](https://www.mparticle.com/static/cc56c80351f32c7c0483e40e68d07cc9/1edee/c96e104d-635b-4b4b-993f-a8f40d39944d_image7.webp) 

#### Connect all sources

Now I need to connect the Web as my inputs: Web to Kinesis.

![mParticle Kinesis Setup2](https://www.mparticle.com/static/327df69aee40633519d9284afbdc0317/54f60/907734de-ebf9-4638-b9bb-b9989035629c_image2.webp) 

The settings I need to provide here are the Amazon Region (us-east-1) and the name of my stream.

![mParticle Kinesis Setup3](https://www.mparticle.com/static/16bfd57b43b9c8508d1682fe9cee4994/17faf/e2ddc16e-1b81-40b8-8fd3-7a40adcfd19e_image1.webp) 

#### Set filters

mParticle lets me switch each individual event name on or off for a particular output, like Kinesis. These settings help me save costs by making sure I'm only sending to Kinesis the data that I need to train my ML model. I'm interested in 4 types of commerce events:

Add to cart
Add to wishlist
Purchase
View detail
In my filter settings, I leave these four events on, and turn everything else off.

![mParticle Kinesis Setup4](https://www.mparticle.com/static/326d58989ed57abd73a4a12ca929aa3e/a1b11/100024a6-4cb3-435f-b754-3c6094776206_image8.webp) 

![mParticle Kinesis Setup5](https://www.mparticle.com/static/eddc0b2a09de7fc6d0dd9c1ea8326a6f/0856d/10a0af44-611d-4c27-ba22-a6f80ee271c0_image4.webp)


## Configure Lambda Parameters and Review Code

Before the destination can send events to your Amazon Personalize events tracker, you will need to tell the destination lambda where to send the events.  It looks for an environment variable called 'personalize_tracking_id'.

Let's set that.  Run the following cell to look up the relevant Amazon Personalize tracker from the Personalize workbook.

We can then set the appropriate value in the destination Lambda.

In [ ]:
ssm = boto3.client('ssm')

# First, let's look up the appropriate tracking string
response = ssm.get_parameter(
    Name='retaildemostore-personalize-event-tracker-id'
)

tracking_id = response['Parameter']['Value']

print(tracking_id)

Go to your AWS console tab or window, and select Lambda from the Services menu.

Find the mParticlePersonalizeEventsDestination, and click on it in the list.

![mParticle Events Destination Lambda](images/mparticle/mparticle-find-lambda-function.png)

The source code used for the AWS Lambda can be found here. 

Then, scroll down to the parameters section.

![mParticle Events Destination Lambda](images/mparticle/mparticle-events-lambda-params.png)

The tracking parameter should be set to the tracker from the first workbook.
The MPARTICLE_API_KEY and MPARTICLE_SECRET_KEY should be set to the key generated earlier above.
The campaignArn should be set by getting the ARN generated within AWS Personalize services.

Click the edit button, then paste in the tracking ID, MPARTICLE_API_KEY, MPARTICLE_SECRET_KEY, campaignArn  from the cell above, and click the Redeploy button at the top of the screen.

Take some time to look at the code that this Lambda uses to send events to Personalize.  You can use this code in your own deployment, however you may need to change the event parameters sent to Amazon Personalize depending on the dataset you set up.

```javascript
const AWS = require('aws-sdk');
const JSONBig = require('json-bigint')({ storeAsString: true });
const mParticle = require('mparticle');
const trackingId = process.env.PERSONALISE_TRACKING_ID;
const report_actions = ["purchase", "view_detail", "add_to_cart", "checkout","add_to_wishlist"];
const mp_api_key = process.env.MPARTICLE_API_KEY;
const mp_api_secret = process.env.MPARTICLE_SECRET_KEY;
const campaignARN = process.env.PERSONALIZE_CAMPAIGN_ARN;
const personalizeevents = new AWS.PersonalizeEvents({apiVersion: '2018-03-22'});
const personalizeruntime = new AWS.PersonalizeRuntime({apiVersion: '2018-05-22'});
console.log("ENVIRONMENT VARIABLES:"+trackingId+":"+mp_api_key+":"+mp_api_secret);
const mp_api = new mParticle.EventsApi(new mParticle.Configuration(mp_api_key, mp_api_secret));

var eventList = [];
var mpid;

exports.handler = function (event, context) {

    console.log(event);
    console.log(event.records);
   // const record = event.Records;
    for (const record of event.records) {
        const payload = JSONBig.parse(Buffer.from(record.data, 'base64').toString('ascii'));
        console.log(payload);
        const events = payload.events;
        mpid = payload.mpid;
        var amazonPersonalizeId = mpid;
        if(payload.user_attributes && payload.user_attributes.amazonPersonalizeId)
            amazonPersonalizeId = payload.user_attributes.amazonPersonalizeId;

        var amazonUserId = mpid;
        if(payload.user_identities){
            for (const identityRecord of payload.user_identities)
            {
                if(identityRecord.identity_type==="customer_id")
                    amazonUserId = identityRecord.identity;
            }
        }
        const sessionId = payload.message_id;
        let params = {
            sessionId: sessionId,
            userId: amazonPersonalizeId,
            trackingId: trackingId
        };
        console.log(params);
        // Check for variant and assign one if not already assigned
        var variant_assigned;
        var variant;
        if(payload.user_attributes && payload.user_attributes.ml_variant)
        {
            variant_assigned = Boolean(payload.user_attributes.ml_variant); 
            variant = variant_assigned ? payload.user_attributes.ml_variant : Math.random() > 0.5 ? "A" : "B";
        }
       
        for (const e of events) {
            if (e.event_type === "commerce_event" && report_actions.indexOf(e.data.product_action.action) >= 0) {
                const timestamp = Math.floor(e.data.timestamp_unixtime_ms / 1000);
                const action = e.data.product_action.action;
                const event_id = e.data.event_id;
                const discount = Math.random() > 0.5 ? "Yes" : "No";
                for (const product of e.data.product_action.products) {
                    const obj = {itemId: product.id,discount: discount};

                    if(eventList.length > 10){
                        eventList.shift();
                        
                    }
                    eventList.push({
                        properties: obj,
                        sentAt: timestamp,
                        eventId: event_id,
                        eventType: action
                    });
                }
            }
        }
        if(eventList.length > 10)
        {
            console.log("eventList more than 10");
            console.log(eventList);
            var lastTenRecords = eventList.length / 2;
            eventList = eventList.slice(lastTenRecords);
            console.log("eventList after slice");
            console.log(eventList);
        }
        if (eventList.length > 0) {
            params.eventList = eventList;
            personalizeevents.putEvents(params, function(err, data) {
                if (err) console.log(err, err.stack);
                else {
                    //getProductPersonalization
                    let params = {
                      // Select campaign based on variant
                      campaignArn: campaignARN,
                      numResults: '5',
                      userId: amazonPersonalizeId
                    };
                    personalizeruntime.getRecommendations(params, function(err, data) {
                      if (err) console.log(err, err.stack);
                      else {
                          let batch = new mParticle.Batch(mParticle.Batch.Environment.development);
                          batch.mpid = mpid;
                          let itemList = [];
                          for (let item of data.itemList) {
                              itemList.push(item.itemId);
                          }
                          batch.user_attributes = {};
                          batch.user_attributes.product_recs = itemList;
                          console.log(itemList);
                          // Record variant on mParticle user profile
                          if (!variant_assigned) {
                              batch.user_attributes.ml_variant = variant
                          }
    
                          let event = new mParticle.AppEvent(mParticle.AppEvent.CustomEventType.other, 'AWS Product Personalization Recs Update');
                          event.custom_attributes = {product_recs: itemList.join()};
                          batch.addEvent(event);
                          let mp_callback = function(error, data, response) {
                              if (error) {
                                  console.error(error);
                                } else {
                                  console.log('API called successfully.');
                                }
                              };
                            mp_api.uploadEvents(batch, mp_callback);
                      }
                    });
    
                }
            });
        }
    }

};
```


## Validate that Real-Time Events are Flowing from the Retail Demo Store

You are now ready to send live events to Personalize from the Retail Demo Store.  In order to do this, you will need to enable the mParticle side integration with the Retail Demo Store.  mParticle provides a variety of ways to collect real time events, and a full discussion of how this works is beyond the scope of this document, however the Retail Demo Store represents a fairly typical deployment for most web applications, in that it uses the mParticle Web SDK library, loaded via NPM, to inject their code into the web application.  

To verify if mParticle JS is fully instantiated within the Retail Demo Store, just open developer console of your web browser and type 
```javascript window.mParticle.Identity.getCurrentUser().getMPID()```

You should get the following response:

![mParticle Verification Setup3](images/mparticle/mparticle-verification-step.png)

This ensures that the mParticle library is available at all times for each user action in the user interface of the Retail Demo Store.  Each time a user performs an action in the Retail Demo Store user interface (such as viewing a product, adding a product to a cart, searching, etc.) an event is sent to mParticle with the properties associated with that user’s action.  Note that this code is only loaded if an environment variable is set to define the write key and secret for the mParticle source which will accept events from the Retail Demo Store.  This will become important in a moment. Here is a sample code snippet allow mParticle to create eCommerce product from the Retail Demo Store and generating a view view product detail event from the website.

```javascript
             let productDetails = window.mParticle.eCommerce.createProduct(
                product.name,
                product.id,
                parseFloat(product.price.toFixed(2)),
                1
            );
            window.mParticle.eCommerce.logProductAction(window.mParticle.ProductActionType.ViewDetail, productDetails);
         
```

This allows you to collect data that is relevant to any tool that might need to be kept updated with the latest user behavior.  

## Sending Real-Time Events

Since you have already connected mParticle to Personalize, let’s test this data path by triggering events from the Retail Demo Store user interface which is deployed in your AWS account.

Run the following code to set the SSM parameter that holds the mParticle API and SECRET key.

```
# Set the mParticle write key in the string below, and run this cell.  

# THIS IS ONLY REQUIRED IF YOU DID NOT SET THE MPARTICLE API KEY and SECRET KEY IN YOUR ORIGINAL DEPLOYMENT

import boto3

ssm = boto3.client('ssm')

if mparticle_write_key:
    response = ssm.put_parameter(
        Name='retaildemostore-mParticle-api-key',
        Value='{}'.format(mParticle_write_key),
        Type='String',
        Overwrite=True
    )

print(mParticle_write_key)

if mparticle_secret_key:
    response = ssm.put_parameter(
        Name='retaildemostore-mParticle-secret-key',
        Value='{}'.format(mParticle_secret_key),
        Type='String',
        Overwrite=True
    )

print(mParticle_secret_key)
```

You now have an environment variable that will enable the mParticle data collection library in the code in `AnalyticsHandler.js`.  All we need to do now, is force a re-deploy of the Retail Demo Store.  

To do that, go back to your AWS Console tab or window, and select Code Pipeline from the Services search.  Then, find the pipeline name that contains `WebUIPipeline` and click that link.

Then, select the ‘Release Change’ button, and confirm the release once the popup shows up.  You will see a confirmation that the pipeline is re-deploying the web ui for the Retail Demo Store.

This process should complete in a few minutes.  Once this is done, you will see the bottom tile confirm that your deployment has completed:



## Log in as a Retail Demo Store User

**IMPORTANT**

Once this is confirmed, tab back to the Retail Demo Store web UI, and refresh the screen to reload the user interface.  This will load the libraries you just deployed, and will allow your instance of the Retail Demo Store to send events to mParticle.

In the Personalize workshop, you created an account for the Retail Demo Store.  If you are not logged in as that account, log in now.

Then, open another tab to the mParticle console, and select Live Stream under Data Master:

![mParticle Livestream View](images/mparticle/mparticle-live-stream.png)

You should see events collected by the mParticle SDK being streamed real-time within your mParticle instance. Feel free to view the events and see the actualy information hold per each event.

## Additional Capability

### Save AWS Personalize Recommended Products back to mParticle

Aside from just sending events from the AWS Retail Demo Store to mParticle, the Lambda function above also sends the commerce events to AWS Personalize. This allows AWS Personalize to receive specific commerce events made by a anonymous and known user and from there allow AWS Personalize to do its magic by providing product recommendation information back to mParticle. Once AWS Personalize has finished computing the relevant products that is associated to the recent events the customer has made, the said product recommendation information will be sent back to mParticle using the mParticle NodeJS SDK. The said code snippet below will set the product_recommendation information as a user attribute (product_recs) within the user's profile. 

```javascript
if (eventList.length > 0) {
            params.eventList = eventList;
            personalizeevents.putEvents(params, function(err, data) {
                if (err) console.log(err, err.stack);
                else {
                    //getProductPersonalization
                    let params = {
                      // Select campaign based on variant
                      campaignArn: campaignARN,
                      numResults: '5',
                      userId: amazonPersonalizeId
                    };
                    personalizeruntime.getRecommendations(params, function(err, data) {
                      if (err) console.log(err, err.stack);
                      else {
                          let batch = new mParticle.Batch(mParticle.Batch.Environment.development);
                          batch.mpid = mpid;
                          let itemList = [];
                          for (let item of data.itemList) {
                              itemList.push(item.itemId);
                          }
                          batch.user_attributes = {};
                          batch.user_attributes.product_recs = itemList;
                          console.log(itemList);
                          // Record variant on mParticle user profile
                          if (!variant_assigned) {
                              batch.user_attributes.ml_variant = variant
                          }
    
                          let event = new mParticle.AppEvent(mParticle.AppEvent.CustomEventType.other, 'AWS Product Personalization Recs Update');
                          event.custom_attributes = {product_recs: itemList.join()};
                          batch.addEvent(event);
                          let mp_callback = function(error, data, response) {
                              if (error) {
                                  console.error(error);
                                } else {
                                  console.log('API called successfully.');
                                }
                              };
                            mp_api.uploadEvents(batch, mp_callback);
                      }
                    });
    
                }
```

![mParticle product_recs](images/mparticle/mparticle-product_recs.png)

### Enrich the Product Recs information from AWS Personalize to show meaningful data in mParticle

mParticle allows a developer to do any data transformations during runtime. This capability is provided by mParticle through Rules. mParticle’s Rules are JavaScript functions that manipulate an incoming batch object from an mParticle input. See mParticle’s Platform Guide for help setting up rules in the mParticle dashboard. - https://docs.mparticle.com/developers/rules

To make the product recommendation returned by AWS Personalize be readable and meaningful in mParticle, what we want to do is to apply mParticle Rules during run-time such that the product IDs are enriched into product names instead. To do that, you'll need to host an additional Lambda function within your AWS environment wherein this Lambda function will have the following code snippet:

```javascript
const https = require('https');

/**
 * Pass the data to send as `event.data`, and the request options as
 * `event.options`. For more information see the HTTPS module documentation
 * at https://nodejs.org/api/https.html.
 *
 * Will succeed with the response body.
 */
const req = require('request');
const axios = require('axios');

exports.handler = async (batch, context, callback) => {

    console.log(batch.user_attributes);
    console.log(batch.user_attributes['product_recs']);
    if (batch.user_attributes['product_recs'] != null) {
        console.log("inside IF");
        //http://retai-loadb-1l2ygk2q7a0ao-2039082906.us-east-1.elb.amazonaws.com/products/id/e850acc3-3cb1-499b-bea9-fd495d4c56ca
        //axios
        var productRecList = batch.user_attributes['product_recs'];
        var productNameList = [];
        console.log(productRecList);
        for (var i = 0; i < productRecList.length; i++) {
            var item = productRecList[i];
            console.log(item);
            var url = "http://retai-loadb-1l2ygk2q7a0ao-2039082906.us-east-1.elb.amazonaws.com/products/id/" + item; //replace this with your retail store product service URL
            var name = await axios.get(url)
                .then(function (resp) {
                    //console.log(response);
                    var data = resp.data.name;
                    console.log(data);
                    return data;
                })
                .catch(function (error) {
                    //   response.send("Error");
                    console.log(error);
                });
            productNameList.push(name);
        }


        batch.user_attributes['product_recs_name'] = productNameList;

    }
    callback(null, batch);


};

```

Once you have deployed the script above in your Lambda function, you'll just have to guide here - https://docs.mparticle.com/guides/platform-guide/rules/ such that the said Lambda function can be accesible within your mParticle environment. Here are some screenshots on how the Lambda rule is applied to the connection.

![mParticle rules_lambda](images/mparticle/mparticle-rules-lambda.png)

Here is the resulting behaviour once Rules has been configured.


![mParticle product_recs](images/mparticle/mparticle-product_recs_enriched.png)

